In [111]:
#libraries
from bs4 import BeautifulSoup, element
import urllib
import requests
import pandas as pd
import numpy as np
import lyricsgenius
import re

#NLP stuff
#Tokenization
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords #look into if I should keep this or not
from nltk.sentiment import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer


In [148]:
#store lyrics from song in this list, for now it will start fresh every time
lyric_list = []

In [179]:
genius = lyricsgenius.Genius("rPVs-pFT7GhBfTxhpu-ISnNCcvCsbRt8wIwhkkEXovrADgSBQfkndQW7Ge22R5Ts")
song = genius.search_song("Fantasy", "Mariah Carey")
lyrics = song.lyrics
print(lyrics)

Searching for "Fantasy" by Mariah Carey...
Done.
53 ContributorsFantasy LyricsThe lead single for Mariah’s Daydream album, this R&B song topped the US charts for eight consecutive weeks, and gained strong success overseas as well. The song was also received positively by critics, who… Read More [Intro]
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-do
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-do
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-do
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-do

[Verse 1]
Oh, when you walk by every night
Talking sweet and looking fine
I get kind of hectic inside
Mmm, baby, I'm so into you
Darling, if you only knew
All the things that flow through my mind

[Chorus]
But it's just a sweet, sweet fantasy, baby
When I close my eyes, you come and take me
On and on and on, it's so deep in my daydreams
But it's just a sweet, sweet fantasy, baby (Fantasy)
(And I want you so badly)

[Post-Chorus]
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-do
Sha-do-do-do-do
Sha-do-do-do-do-do-do-do-d

# Cleaning Lyrics

In [180]:
#Data Cleaning/Processing

def clean_lyrics(raw_lyrics):
    #Get rid of information shoved at beginning of string leaving only the lyrics of the song
    starter = ']' #This always finds the first closed bracket which is always the closing bracket for '[Intro]'
    position = raw_lyrics.find(starter)
    lyrics = raw_lyrics[position+1:]

    # Remove section headers like [Chorus], [Verse 1], etc.
    lyrics = re.sub(r"\[.*?\]", "", lyrics)

    # Remove extra whitespace, newlines
    lyrics = re.sub(r"\n+", " ", lyrics)
    lyrics = re.sub(r"\s+", " ", lyrics)

    # Optionally remove punctuation
    lyrics = re.sub(r"[^\w\s']", "", lyrics)

    # Lowercase
    lyrics = lyrics.lower()

    return lyrics


In [181]:
cleaned_lyrics = clean_lyrics(lyrics)
print(cleaned_lyrics)

 shadodododo shadodododododododo shadodododo shadodododododododo shadodododo shadodododododododo shadodododo shadodododododododo oh when you walk by every night talking sweet and looking fine i get kind of hectic inside mmm baby i'm so into you darling if you only knew all the things that flow through my mind but it's just a sweet sweet fantasy baby when i close my eyes you come and take me on and on and on it's so deep in my daydreams but it's just a sweet sweet fantasy baby fantasy and i want you so badly shadodododo shadodododododododo shadodododo shadodododododododo shadodododo shadodododododododo shadodododo shadodododododododo images of rapture creep into me slowly as you're going to my head and my heart beats faster when you take me over time and time and time again but it's just a sweet sweet fantasy baby when i close my eyes you come and take me on and on and on it's so deep in my daydreams but it's just a sweet sweet fantasy baby fantasy oh oh it's just a sweet sweet fantasy 

# Tokenization/Lemmatization/Filtering stopwords

In [115]:
# Tokenize into words
words = word_tokenize(cleaned_lyrics)

# Tokenize into sentences
sentences = sent_tokenize(cleaned_lyrics) #this one doesn't do anything 

In [116]:
print(words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'dreams', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'was', 'mine', 'i', "'d", 'take', 'your', 'enemies', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 

In [117]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

In [118]:
print(lemmatized_words)

['if', 'this', 'world', 'were', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'it', 'like', 'it', "'s", 'hot', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'dream', 'and', 'make', "'em", 'multiply', 'if', 'this', 'world', 'wa', 'mine', 'i', "'d", 'take', 'your', 'enemy', 'in', 'front', 'of', 'god', 'introduce', "'em", 'to', 'that', 'light', 'hit', 'them', 'strictly', 'with', 'that', 'fire', 'it', "'s", 'a', 'vibe', 'do', 'your', 'dance', 'let', "'em", 'watch', 'she', 'a', 'fan', 'he', 'a', 'flop', 'they', 'just', 'wan', 'na', 'kumbaya',

In [119]:
#Consider if I should keep this or not. Lose of some words may affect models effectivness
stop_words = set(stopwords.words("english"))

filtered_words = [word for word in lemmatized_words if word not in stop_words]

In [120]:
print(filtered_words)

['world', 'mine', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', 'fahfah', 'fahfahfah', 'fahfah', 'fah', 'hey', 'roman', 'numeral', 'seven', 'bae', 'drop', 'like', "'s", 'hot', 'world', 'wa', 'mine', "'d", 'take', 'dream', 'make', "'em", 'multiply', 'world', 'wa', 'mine', "'d", 'take', 'enemy', 'front', 'god', 'introduce', "'em", 'light', 'hit', 'strictly', 'fire', "'s", 'vibe', 'dance', 'let', "'em", 'watch', 'fan', 'flop', 'wan', 'na', 'kumbaya', 'nah', 'world', 'concrete', 'flower', 'grow', 'heartache', 'doin', "'", 'know', 'weekend', 'get', 'poppin', "'", 'low', 'better', 'day', 'comin', "'", 'sure', 'world', 'wa', 'would', "n't", 'give', 'nobody', 'sympathy', "'d", 'take', 'away', 'pain', "'d", 'give', 'everything', 'wan', 'na', 'see', 'win', 'wan', 'na', 'see', 'world

# NLP

In [182]:
lyric_list.append(cleaned_lyrics)

In [183]:
print(lyric_list)

[" if this world were mine hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire fahfah fahfahfah fahfah fah hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire it's a vibe do your dance let 'em watch she a fan he a flop they just wanna kumbaya nah in this world concrete flowers grow heartache she only doin' what she know weekends get it poppin' on the low better days comin' for sure if this world were if it was up to me i wouldn't give these nobodies no sympathy i'd take away the pain i'd give you everything i just wanna see you win wanna see if this world were mine it go in when you out ride it do it real slow slide baby

In [184]:
print(len(lyric_list))

5


In [185]:
print(lyric_list[0])

 if this world were mine hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire fahfah fahfahfah fahfah fah hey roman numeral seven bae drop it like it's hot if this world was mine i'd take your dreams and make 'em multiply if this world was mine i'd take your enemies in front of god introduce 'em to that light hit them strictly with that fire it's a vibe do your dance let 'em watch she a fan he a flop they just wanna kumbaya nah in this world concrete flowers grow heartache she only doin' what she know weekends get it poppin' on the low better days comin' for sure if this world were if it was up to me i wouldn't give these nobodies no sympathy i'd take away the pain i'd give you everything i just wanna see you win wanna see if this world were mine it go in when you out ride it do it real slow slide baby y

In [186]:
#Sentiment Analysis
sia = SentimentIntensityAnalyzer()
scores = sia.polarity_scores(cleaned_lyrics)

print(scores)
# {'neg': 0.0, 'neu': 0.511, 'pos': 0.489, 'compound': 0.5719}

{'neg': 0.027, 'neu': 0.667, 'pos': 0.307, 'compound': 0.9988}


In [187]:
#Determine setniment score based off compount

if scores['compound'] >= 0.05:
    sentiment = 'positive'
elif scores['compound'] <= -0.05:
    sentiment = 'negative'
else:
    sentiment = 'neutral'

In [188]:
print(sentiment)

positive


In [193]:
#Identify which words are most unique
"""tfidf_matrix needs lyrics from several songs to properly score lyrics"""

# Assume `lyrics_list` is a list of cleaned lyrics from multiple songs
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
tfidf_matrix = vectorizer.fit_transform(lyric_list)

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Top keywords for a song:

def get_top_keywords(tfidf_vector, feature_names, top_n=10):
    vector_dense = tfidf_vector.toarray().flatten()
    sorted_indices = np.argsort(vector_dense)[::-1]  # Descending order
    top_keywords = [(feature_names[i], vector_dense[i]) for i in sorted_indices[:top_n] if vector_dense[i] > 0]
    return top_keywords
    # sorted_indices = np.argsort(tfidf_vector.toarray()[0])[::-1]
    # top_keywords = [feature_names[i] for i in sorted_indices[:top_n]]
    # return top_keywords

In [194]:
get_top_keywords(tfidf_matrix[0], feature_names)

[('world', 0.5510086154289483),
 ('wanna', 0.22958692309539513),
 ('em', 0.22958692309539513),
 ('days', 0.1836695384763161),
 ('better', 0.1836695384763161),
 ('comin', 0.1836695384763161),
 ('turn', 0.14818342716523641),
 ('sure', 0.13775215385723708),
 ('poppin', 0.13775215385723708),
 ('low', 0.13775215385723708)]

In [191]:
print(tfidf_matrix[0])

  (0, 178)	0.04591738461907902
  (0, 198)	0.04591738461907902
  (0, 226)	0.04591738461907902
  (0, 250)	0.04591738461907902
  (0, 141)	0.04591738461907902
  (0, 196)	0.03075140762596771
  (0, 145)	0.13775215385723708
  (0, 255)	0.14818342716523641
  (0, 242)	0.043759759596395595
  (0, 266)	0.04591738461907902
  (0, 273)	0.09183476923815805
  (0, 252)	0.037045856791309104
  (0, 129)	0.04591738461907902
  (0, 142)	0.07409171358261821
  (0, 108)	0.037045856791309104
  (0, 268)	0.04591738461907902
  (0, 177)	0.04591738461907902
  (0, 236)	0.037045856791309104
  (0, 51)	0.04591738461907902
  (0, 186)	0.037045856791309104
  (0, 80)	0.04591738461907902
  (0, 223)	0.04591738461907902
  (0, 277)	0.04591738461907902
  (0, 219)	0.04591738461907902
  (0, 30)	0.04591738461907902
  :	:
  (0, 128)	0.037045856791309104
  (0, 34)	0.04591738461907902
  (0, 258)	0.04591738461907902
  (0, 64)	0.04591738461907902
  (0, 66)	0.04591738461907902
  (0, 65)	0.09183476923815805
  (0, 227)	0.09183476923815805
  (

In [192]:
print(feature_names)

['alive' 'anymore' 'attack' 'away' 'baby' 'bad' 'badly' 'bae' 'beats'
 'beginning' 'better' 'bite' 'blinded' 'bone' 'bored' 'boyfriend'
 'calling' 'cause' 'cheap' 'city' 'clearly' 'close' 'cold' 'come' 'comin'
 'concrete' 'control' 'count' 'crawl' 'creep' 'crown' 'cube' 'cuts'
 'dadada' 'dance' 'darling' 'date' 'day' 'daydreams' 'days' 'deafening'
 'deep' 'deserve' 'didn' 'doin' 'don' 'door' 'downset' 'dream' 'dreaming'
 'dreams' 'drip' 'drop' 'drowning' 'drums' 'earth' 'easy' 'em' 'end'
 'enemies' 'enjoy' 'eye' 'eyes' 'fab' 'fah' 'fahfah' 'fahfahfah' 'fan'
 'fantasy' 'faster' 'feel' 'feels' 'fine' 'fingernails' 'flop' 'flow'
 'flowers' 'fly' 'forgotten' 'friends' 'froze' 'fuck' 'fun' 'gets' 'god'
 'goes' 'goin' 'going' 'gone' 'gonna' 'good' 'got' 'gotta' 'grow' 'guitar'
 'handle' 'hard' 'having' 'head' 'hearing' 'heart' 'heartache'
 'heartbeats' 'heaven' 'hectic' 'hey' 'hit' 'hold' 'home' 'hope' 'horse'
 'hot' 'hotels' 'images' 'innocent' 'inside' 'introduce' 'isn' 'judge'
 'just' 'ke